Notebook imitant:
    https://towardsdatascience.com/causal-machine-learning-for-econometrics-causal-forests-5ab3aec825a7

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn 

In [2]:
#Read main data to DataFrame
df = pd.read_stata('Cash_and_Childhood_Development_Replication/macoursetal_main.dta')  

In [3]:
# "hogarid_old","cp_old","unique_05", "cpmom_06" seran usats com a index
df_out=df[["a5sscore_i_06","a6smemory_p1_06","a6smemory_p2_06","a7a_delay_06","a7b_delay_06",
        "a7c_delay_06","a7d_delay_06","a9sgrossmotor_06","height_06","weight_06","z_tvip_06","z_social_06",
        "z_language_06","z_finmotor_06","z_memory_06","z_grmotor_06","z_legmotor_06","z_behavior_06","z_height_06",
        "z_weight_06","z_all_06"]]


# Income variables

#2-6: Treiem variables:

"hogarid_old","cp_old", "unique_05", "itt_all_i", "TREAT1","TREAT2","TREAT3","TREAT4", "com_tvip_05", "com_control_05"
, "com_notvip", "sample06","weighted_05", ,"itt_i"

In [4]:
df1=df[["hogarid_old","s1age_head_05","s1hhsize_05","s1hhsz_undr5_05","s1hhsz_5_14_05",
        "s1hhsz_15_24_05","s1hhsz_25_64_05","s1hhsz_65plus_05","s1male_head_05","s2mother_inhs_05","s3ap5_rooms_h_05",
        "s3ap23_stime_h_05","s3ap24_htime_h_05","s3ap25_hqtime_h_05","s3atoilet_hh_05","s3awater_access_hh_05",
        "s3aelectric_hh_05","s4p6_vitamina_i_05","s4p7_parasite_i_05","s11ownland_hh_05","cons_food_pc_05","cons_tot_pc_05",
        "height_05","a10whz_05","weight_05","yrsedfath","age_transfer","bweight",
        "s4p7_parasite_i_06","T","male","ed_mom","MUN1","MUN2","MUN3","MUN4",
        "MUN5","MUN6","com_haz_05","com_waz_05","com_vit_05","com_deworm_05",
        "vitamiron_06", "propfood_05","prstap_f_05","pranimalprot_f_05","prfruitveg_f_05"]]

In [5]:
df1.rename(columns = {'T':'tr'}, inplace = True)

C:\Users\Usuari\AppData\Local\Temp\ipykernel_8012\2300806653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns = {'T':'tr'}, inplace = True)


In [6]:
#fem una copia del df
def copy_df(df):
   return df.copy()

In [7]:
#eliminem individus amb >delta*100% columnes buides
def drop_ind_missing(df, delta):
    thresh = len(df.columns)*delta
    df.dropna(axis=0, thresh=thresh, inplace=True)
    return df

In [8]:
#eliminem individus amb age transfer<age
def drop_out_missing(df, age=-11):
    nan_rows=df[df.age_transfer<age].index
    df.drop(nan_rows, inplace=True)
    return df

In [9]:
#categorical to ordinal
def ordinal(df, cols):
    enc = OrdinalEncoder()
    enc.fit(df[cols])
    df[cols] = enc.transform(df[cols])
    
    return df

In [10]:
#imputem NaNs

#imputem mitjanes
def mean_imputer(df, cols):
    for col in cols:
        df[col].fillna(df[col].mean(), inplace=True)
    return df

#imputem el valor més frequent de la columna dintre el hh
def hh_mf_imputer(df,cols):
    index=df.index
    for col in cols:
        col_hh=[df[df.hogarid_old==familia][[col,"age_transfer"]].sort_values(by=['age_transfer'])[col]
                .values for familia in df.hogarid_old]
        for i in range(len(index)):
               if np.isnan(df[col][index[i]]):
                    non_nan_len=np.count_nonzero(~np.isnan(col_hh[i]))
                    if(non_nan_len!=0):
                        vals,counts = np.unique(col_hh[i], return_counts=True)
                        ind = np.argmax(counts)
                        df.loc[index[i],col]=vals[ind]
    return df             
                            
                        

In [11]:
#eliminem outliers

def outlier_removal(df, cols):
    outlier_threshold = []
    for col in cols:
            q3 = np.nanpercentile(df[col], 75)
            q1 = np.nanpercentile(df[col], 25)
            iqr = q3 - q1
            out_low = q1 - 3*iqr
            out_high = q3 + 3*iqr
            outlier_threshold.append([out_low, out_high])
    outliers_indexs=[]
    for col,k in zip(cols,range(len(cols))):
        #outliers_indexs_in_col=[]
        for i in df[col].index:
            if df[col][i]<outlier_threshold[k][0] or df[col][i]>outlier_threshold[k][1]:
                outliers_indexs.append(i)
        #outliers_indexs.append(outliers_indexs_in_col)
    return df.drop(list(set(outliers_indexs)), axis=0)

In [12]:
def normalize(df, cols):
    Nscaler = MinMaxScaler()

    Nscaler.fit(df[cols])
    df[cols] = Nscaler.transform(df[cols])
    
    return df

def standardize(df, cols):
    Sscaler = StandardScaler()

    Sscaler.fit(df[cols])
    df[cols] = Sscaler.transform(df[cols])
    
    return df

In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

mean_cols=["bweight", "height_05", "a10whz_05", "weight_05"]
hh_mf_cols=["s1age_head_05","s1hhsize_05","s1hhsz_undr5_05","s1hhsz_5_14_05","s1hhsz_15_24_05","s1hhsz_25_64_05",
            "s1hhsz_65plus_05","s1male_head_05","s2mother_inhs_05","s3ap5_rooms_h_05","s3atoilet_hh_05",
            "s3awater_access_hh_05","s3aelectric_hh_05","s11ownland_hh_05","s4p7_parasite_i_06","ed_mom","vitamiron_06",
           "s4p6_vitamina_i_05", "s4p7_parasite_i_05", "cons_food_pc_05", "yrsedfath", "propfood_05", "prstap_f_05"
            ,  "pranimalprot_f_05",  "prfruitveg_f_05"]

categorical_cols=df1.select_dtypes(exclude=["number","bool_","object_"]).columns
outlier_cols=["pranimalprot_f_05", "bweight", "prfruitveg_f_05"]

normal_cols =["height_05","a10whz_05","weight_05","com_haz_05","com_waz_05"]

Nnormal_cols = ['s1age_head_05', 's3ap23_stime_h_05', 's3ap24_htime_h_05', 's3ap25_hqtime_h_05', 'cons_food_pc_05', 
                'cons_tot_pc_05', 'yrsedfath','age_transfer', 'bweight', 'ed_mom', 'com_tvip_05', 'com_control_05']


v=(df1.pipe(copy_df)
            .pipe(drop_ind_missing, 0.8)
            .pipe(drop_out_missing)
            .pipe(ordinal, categorical_cols)
            .pipe(hh_mf_imputer, hh_mf_cols)
            .pipe(mean_imputer, mean_cols+hh_mf_cols)
            .pipe(outlier_removal, outlier_cols)) #outliers
            #.pipe(standardize, normal_cols) #estandaritzacio
            #.pipe(normalize, Nnormal_cols)) #normalitzacio
                         

In [14]:
cleaned_df=v.copy()

In [15]:
cleaned_df.isna().sum()

hogarid_old              0
s1age_head_05            0
s1hhsize_05              0
s1hhsz_undr5_05          0
s1hhsz_5_14_05           0
s1hhsz_15_24_05          0
s1hhsz_25_64_05          0
s1hhsz_65plus_05         0
s1male_head_05           0
s2mother_inhs_05         0
s3ap5_rooms_h_05         0
s3ap23_stime_h_05        0
s3ap24_htime_h_05        0
s3ap25_hqtime_h_05       0
s3atoilet_hh_05          0
s3awater_access_hh_05    0
s3aelectric_hh_05        0
s4p6_vitamina_i_05       0
s4p7_parasite_i_05       0
s11ownland_hh_05         0
cons_food_pc_05          0
cons_tot_pc_05           0
height_05                0
a10whz_05                0
weight_05                0
yrsedfath                0
age_transfer             0
bweight                  0
s4p7_parasite_i_06       0
tr                       0
male                     0
ed_mom                   0
MUN1                     0
MUN2                     0
MUN3                     0
MUN4                     0
MUN5                     0
M

In [16]:
categorical_cols

Index(['s1male_head_05', 's3atoilet_hh_05', 's3awater_access_hh_05',
       's3aelectric_hh_05', 's4p6_vitamina_i_05', 's4p7_parasite_i_05',
       's11ownland_hh_05', 's4p7_parasite_i_06'],
      dtype='object')

In [17]:
treatment = 'tr'
outcome = 'z_all_06'
covariates = ["s1age_head_05","s1hhsize_05","s1hhsz_undr5_05","s1hhsz_5_14_05",
        "s1hhsz_15_24_05","s1hhsz_25_64_05","s1hhsz_65plus_05","s1male_head_05","s2mother_inhs_05","s3ap5_rooms_h_05",
        "s3ap23_stime_h_05","s3ap24_htime_h_05","s3ap25_hqtime_h_05","s3atoilet_hh_05","s3awater_access_hh_05",
        "s3aelectric_hh_05","s4p6_vitamina_i_05","s4p7_parasite_i_05","s11ownland_hh_05","cons_food_pc_05","cons_tot_pc_05",
        "height_05","a10whz_05","weight_05","yrsedfath","age_transfer","bweight",
        "s4p7_parasite_i_06","male","ed_mom","MUN1","MUN2","MUN3","MUN4",
        "MUN5","MUN6","com_haz_05","com_waz_05","com_vit_05","com_deworm_05",
        "vitamiron_06", "propfood_05","prstap_f_05","pranimalprot_f_05","prfruitveg_f_05"]

In [18]:
cleaned_df.drop(cleaned_df[df_out.iloc[cleaned_df.index].z_all_06.isna()].index, axis=0, inplace=True)

In [19]:
cleaned_df["z_all_06"]=df_out.iloc[cleaned_df.index].z_all_06

In [20]:
cp_cols=["hogarid_old"]

In [21]:
cleaned_df.drop(cp_cols, axis=1, inplace=True)

In [22]:
from sklearn.model_selection import train_test_split
# split data into train and test sets 
train, test = train_test_split(cleaned_df, test_size=0.2)

# set variables for causal forest Y=outcome, T=treatment, X=covariates, W=effect_modifiers 
Y = train[outcome]
T = train[treatment]
X = train[covariates]
W = None
X_test = test[covariates]

In [23]:
from econml.dml import CausalForestDML
from sklearn.linear_model import LassoCV

# set parameters for causal forest 
causal_forest1 = CausalForestDML(criterion='het', 
                                n_estimators=10000,       
                                min_samples_leaf=10, 
                                max_depth=None, 
                                max_samples=0.5,
                                discrete_treatment=False,
                                honest=True,
                                inference=True,
                                cv=10,
                                model_t=LassoCV(), 
                                model_y=LassoCV(),
                                )
                      
# fit train data to causal forest model 
causal_forest1.fit(Y, T, X=X, W=W)
# estimate the CATE with the test set 
causal_forest1.const_marginal_ate(X_test)

In [24]:
a=causal_forest1.effect(X)

Mirem de interpretar i identificar subgrups a través de les eines d'interpretació de econML. Primer fem un decision tree naive, sembla que fa el mateix que SingleTreeCateInterpreter. Després provem aquest (SingleTreeCateInterpreter) i llavors SingleTreePolicyInterpreter

In [36]:
from econml.cate_interpreter import SingleTreeCateInterpreter
from econml.dml import LinearDML
import graphviz
import os 
os.environ["PATH"] += os.pathsep + 'C:\Program Files\Graphviz/bin/'

intrp = SingleTreeCateInterpreter(include_model_uncertainty=True, max_depth=4, min_samples_leaf=100)
# We interpret the CATE model's behavior based on the features used for heterogeneity
intrp.interpret(causal_forest1, X)
# Plot the tree
intrp.plot(feature_names=covariates, fontsize=12)
intrp.plot()


Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.


In [37]:
intrp.render("prova1", feature_names=covariates)

In [27]:
from econml.cate_interpreter import SingleTreePolicyInterpreter

intrp = SingleTreePolicyInterpreter(risk_level=None, max_depth=2, min_samples_leaf=1,min_impurity_decrease=.001)
intrp.interpret(causal_forest1, X, sample_treatment_costs=0.02)


In [28]:
intrp.render("prova2", feature_names=covariates)

### SHAP

In [29]:
pd.Series(causal_forest1.feature_importances_,index=causal_forest1.cate_feature_names()).sort_values()

MUN2                     9.632466e-07
MUN3                     3.301575e-04
s1hhsz_65plus_05         8.653889e-04
s1male_head_05           9.806152e-04
MUN4                     1.330209e-03
MUN1                     1.807067e-03
s3aelectric_hh_05        2.376285e-03
MUN6                     2.605222e-03
vitamiron_06             2.763962e-03
s4p7_parasite_i_06       2.806390e-03
s3atoilet_hh_05          2.847659e-03
MUN5                     3.178768e-03
s1hhsz_25_64_05          3.283734e-03
s3awater_access_hh_05    3.425839e-03
s11ownland_hh_05         4.667341e-03
s3ap5_rooms_h_05         5.263231e-03
s4p6_vitamina_i_05       5.310475e-03
s1hhsz_undr5_05          5.624611e-03
s4p7_parasite_i_05       5.946280e-03
s1hhsz_15_24_05          7.559086e-03
male                     9.892142e-03
s1hhsz_5_14_05           1.136967e-02
s1hhsize_05              1.421914e-02
s3ap25_hqtime_h_05       1.750036e-02
yrsedfath                1.923727e-02
s3ap24_htime_h_05        2.319922e-02
ed_mom      

In [30]:
import shap
from econml.dml import CausalForestDML

# fit causal forest with default parameters 
causal_forest = CausalForestDML()
causal_forest.fit(Y, T, X=X, W=W)

# calculate shap values of causal forest model 
shap_values = causal_forest.shap_values(X)


'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please

In [31]:

# plot shap values 
shap.summary_plot(shap_values['z_all_06']["tr"])

Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
